In [1]:
import pandas as pd
import plotly.express as px
import jalali_pandas
import numpy as np

In [2]:
excel_file = pd.ExcelFile("data.xlsx")
data = pd.read_excel('data.xlsx')

In [3]:
excel_file.sheet_names

['Table2',
 'Sheet2',
 'قیمت دستور کار مهر',
 'قیمت دستورکار شهریورماه',
 'بانک ماشین آلات',
 'Sheet3',
 'تعمیرات2.4 الی 2.6',
 'تعمیرات2.2 الی 2.4',
 'تعمیرات12.1 الی 2.2',
 'تعمیرات10.1 الی 12.1',
 'تعمیرات8.1 الی 10.1',
 'تعمیرات 6.31 الی8.1',
 'Sheet1',
 'مرداد -قیمت',
 'تیر - قیمت',
 'خرداد-قیمت',
 'اردیبهشت-قیمت',
 'فروردین-قیمت',
 'اسفند-قیمت',
 'بهمن-قیمت',
 'دی قیمت',
 'آذر-قیمت',
 'آبان - قیمت',
 'مهر1403-قیمت',
 'تعمیرات2.6 الی 3.8',
 'بانک تعمیرات']

### Tamirat Data Processing

In [4]:
tamirat_list_name = [name for name in excel_file.sheet_names if "الی" in name.lower()]

In [5]:
tamirat_data = pd.DataFrame()
for name in tamirat_list_name:
    df = pd.read_excel(excel_file, sheet_name=name)
    df['i'] = name
    tamirat_data = pd.concat([tamirat_data, df]).fillna('-')

In [6]:
tamirat_data.columns = tamirat_data.iloc[2].to_list()
tamirat_data.drop(index=[0, 1, 2], inplace=True)

In [7]:
px.bar(tamirat_data['Main system'].value_counts())

In [8]:
tamirat_data['sub system'].value_counts().to_frame().to_excel('cleaned_data/count_subsystem.xlsx')

### Price and Open-Close Analysis

In [9]:
price_sheetname_list = [name for name in excel_file.sheet_names if "قیمت" in name]

In [10]:
price_data = pd.DataFrame()
for name in price_sheetname_list:
    df = pd.read_excel(excel_file, sheet_name=name)
    df['i'] = name
    price_data = pd.concat([price_data, df]).fillna('-')

In [11]:
price_data.columns = price_data.iloc[2].to_list()
price_data.drop(index=[0, 1, 2], inplace=True)

In [12]:
price_data['finish'] = np.where(price_data['finish']==0, price_data['start'], price_data['finish'])
price_data['start-g'] = price_data['start'].jalali.parse_jalali("%Y/%m/%d").jalali.to_gregorian()
price_data['finish-g'] = price_data['finish'].jalali.parse_jalali("%Y/%m/%d").jalali.to_gregorian()
price_data['duration'] = (price_data['finish-g'] - price_data['start-g']).dt.days

In [13]:
price_data.sort_values('duration', ascending=False).head(10)

,order number,prise,start,finish,قیمت دستور کار مهر,start-g,finish-g,duration
4119,1397852,4400000,1403/08/12,1404/09/20,آبان - قیمت,2024-11-02,2025-12-11,404
6601,1393375,34400000,1403/12/04,1404/12/28,اسفند-قیمت,2025-02-22,2026-03-19,390
3859,1395179,35000000,1403/08/21,1404/09/14,آبان - قیمت,2024-11-11,2025-12-05,389
6461,1393895,10700000,1403/11/21,1404/11/28,بهمن-قیمت,2025-02-09,2026-02-17,373
785,1452021,450000000,1404/05/29,1405/06/03,مرداد -قیمت,2025-08-20,2026-08-25,370
5024,1398264,1600000,1403/12/07,1404/12/09,اسفند-قیمت,2025-02-25,2026-02-28,368
5857,1323135,376000000,1403/11/30,1404/12/01,بهمن-قیمت,2025-02-18,2026-02-20,367
193,1396782,73500000,1403/12/05,1404/12/05,اسفند-قیمت,2025-02-23,2026-02-24,366
2306,1396382,46400000,1403/12/29,1404/12/29,اسفند-قیمت,2025-03-19,2026-03-20,366
9587,1461443,96000000,1403/09/12,1404/09/12,آذر-قیمت,2024-12-02,2025-12-03,366


### price and tamirat lookup

In [14]:
merged_tamirat_price = tamirat_data.merge(price_data, left_on='order number', right_on='order number', how='inner')
merged_tamirat_price['quarter'] = merged_tamirat_price["Repair date"].jalali.parse_jalali("%Y/%m/%d").jalali.quarter

In [16]:
px.scatter(
    merged_tamirat_price.groupby('Main system').agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False),
    x='Main system',
    y='duration', 
    size='prise',
    color='Work code')

In [ ]:
merged_tamirat_price.groupby(['quarter', 'Main system']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False)

,quarter,Main system,duration,prise,Work code
4,1,بازسازی,16.346154,3.890731e+09,26
16,1,سیستم وکیوم دستگاه جاروب,31.500000,3.666250e+09,2
40,2,سیستم نمک پاش و برفروب,26.974359,2.788828e+09,39
70,3,سیستم وکیوم دستگاه جاروب,0.250000,1.763281e+09,8
50,2,نمکپاش,13.285714,1.744379e+09,7
...,...,...,...,...,...
5,1,تعویض روغن,18.035088,8.459000e+07,570
88,4,سرویس و نگهداری,20.169683,7.849269e+07,442
101,4,معاینه فنی,14.117647,7.454206e+07,34
47,2,معاینه فنی,19.100000,7.058000e+07,10


In [16]:
merged_tamirat_price.groupby(['quarter']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('duration', ascending=False)

,quarter,duration,prise,Work code
3,4,22.904891,338120494.31412,18442
0,1,17.347365,424517997.412812,22259
1,2,13.848077,479979419.814327,22123
2,3,13.212308,336615550.060022,42867


In [17]:
px.scatter(
    merged_tamirat_price.groupby(['quarter', 'Main system']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False),
    x='Main system',
    y='prise', 
    size='duration',
    color='quarter')

### Machines Properties

In [20]:
machine_bank = pd.read_excel('data.xlsx', sheet_name='بانک ماشین آلات')

In [21]:
merged_tamirat_price_bank = merged_tamirat_price.merge(machine_bank, on='Machine code')

In [22]:
px.bar(merged_tamirat_price_bank['UnitGroup'].value_counts())

In [23]:
px.scatter(
    merged_tamirat_price_bank.groupby(['UnitGroup']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False),
    x='UnitGroup',
    y='prise', 
    size='duration',
    color='Work code'
    )

In [24]:
merged_tamirat_price_bank['duration'].isna().sum()

0

In [25]:
merged_tamirat_price_bank.groupby(['CompanyName']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False)

,CompanyName,duration,prise,Work code
82,محبی,100.000000,1.200000e+10,1
109,کیس,32.321429,6.536607e+09,28
8,اشمیتس,3.250000,2.908375e+09,8
97,ویرتگن,25.555556,1.648800e+09,9
25,بوماگ,12.916667,1.454208e+09,12
...,...,...,...,...
6,استریک,4.416667,1.862667e+07,12
42,دلتا,-0.333333,1.296667e+07,3
37,جیپ,4.285714,1.222143e+07,7
111,گراو,0.000000,1.175000e+07,2


In [26]:
merged_tamirat_price_bank =merged_tamirat_price_bank[~merged_tamirat_price_bank['BuildDate'].isna()]
merged_tamirat_price_bank['BuildDate'] = merged_tamirat_price_bank['BuildDate'].astype(int)
px.scatter(
    merged_tamirat_price_bank.groupby(['CompanyName']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count', 'BuildDate': 'mean'}).reset_index().sort_values('BuildDate', ascending=False),
    x='CompanyName',
    y='prise',
    color='BuildDate',
    size = 'Work code')

In [27]:
merged_tamirat_price_bank =merged_tamirat_price_bank[~merged_tamirat_price_bank['quarter'].isna()]
merged_tamirat_price_bank['quarter'] = merged_tamirat_price_bank['quarter'].astype(int)
px.scatter(
    merged_tamirat_price_bank.groupby(['CompanyName']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count', 'quarter': 'mean'}).reset_index().sort_values('Work code', ascending=False),
    x='CompanyName',
    y='quarter',
    color='prise',
    size = 'Work code')

In [28]:
px.bar(merged_tamirat_price_bank['CarType'].value_counts())

In [29]:
merged_tamirat_price_bank.groupby(['CarType']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False)

,CarType,duration,prise,Work code
39,کامیون جاروب,11.309392,1.851501e+09,181
0,آسفالت تراش,25.555556,1.648800e+09,9
40,کامیون جرثقیل,10.593801,1.183830e+09,1226
3,بیل بکهو,13.122807,9.358412e+08,228
14,غلتک چرخ آهنی,6.200000,7.211960e+08,25
...,...,...,...,...
31,ون,0.500000,1.625000e+07,2
63,کفی,5.190476,1.615429e+07,21
23,مینی بیل,24.500000,1.605000e+07,2
26,وانت آتشنشانی,0.000000,1.016000e+07,5


In [30]:
pmerged_tamirat_price_bank =merged_tamirat_price_bank[~merged_tamirat_price_bank['quarter'].isna()]
merged_tamirat_price_bank['quarter'] = merged_tamirat_price_bank['quarter'].astype(int)
px.scatter(
    merged_tamirat_price_bank.groupby(['CarType']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count', 'quarter': 'mean'}).reset_index().sort_values('Work code', ascending=False),
    x='CarType',
    y='prise',
    color='duration',
    size = 'Work code')

In [39]:
pmerged_tamirat_price_bank =merged_tamirat_price_bank[~merged_tamirat_price_bank['quarter'].isna()]
merged_tamirat_price_bank['quarter'] = merged_tamirat_price_bank['quarter'].astype(int)
px.scatter(
    merged_tamirat_price_bank.groupby('CarType').agg({
    'duration': 'max',
    'prise': 'mean',
    'Work code': 'count',
    'quarter': lambda x: x.mode().iloc[0] if not x.mode().empty else None}).reset_index().sort_values('Work code', ascending=False),
    x= 'CarType',
    y='prise',
    color='quarter',
    size = 'duration')

In [40]:
merged_tamirat_price_bank.groupby(['Repairman']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count'}).reset_index().sort_values('prise', ascending=False)

,Repairman,duration,prise,Work code
3531,کارن صنعت کاوه,0.0,2.419784e+10,28
720,تاجریزی(نمایندگی بازرگانی ایران توربو),47.0,1.240294e+10,5
2518,علی مرادزاده اسکندری,47.0,1.240294e+10,5
10,مهران کمپرس,29.9,1.109648e+10,10
2140,شرکت پایارشدکوشا,5.0,7.569000e+09,3
...,...,...,...,...
1172,تعویض روغنی و پنچر گیری جلال کریمی,0.0,7.000000e+05,1
1563,حمید دامغانی,0.0,6.500000e+05,1
144,آپاراتی و تعویض روغن مهدی,1.0,5.000000e+05,1
1937,ساداتی,0.0,4.000000e+05,1


In [52]:
pmerged_tamirat_price_bank =merged_tamirat_price_bank[~merged_tamirat_price_bank['quarter'].isna()]
merged_tamirat_price_bank['quarter'] = merged_tamirat_price_bank['quarter'].astype(int)
px.scatter(
    merged_tamirat_price_bank.groupby(['Repairman']).agg({'duration': 'mean', 'prise': 'mean', 'Work code': 'count', 'quarter': 'mean'}).reset_index().sort_values('prise', ascending=False),
    x='Repairman',
    y='prise',
    # color='duration',
    size = 'Work code')


In [55]:
pmerged_tamirat_price_bank =merged_tamirat_price_bank[~merged_tamirat_price_bank['quarter'].isna()]
merged_tamirat_price_bank['quarter'] = merged_tamirat_price_bank['quarter'].astype(int)
merged_tamirat_price_bank['duration'] = merged_tamirat_price_bank['duration'].astype(int)
merged_tamirat_price_bank['duration'] = merged_tamirat_price_bank['duration'].abs()
copy_merged_for_plot = merged_tamirat_price_bank.copy()
copy_merged_for_plot = copy_merged_for_plot[copy_merged_for_plot['Repairman']!='*']
px.scatter(
    copy_merged_for_plot.groupby(['Repairman']).agg({'duration': 'sum', 'prise': 'mean', 'Work code': 'count', 'quarter': 'max'}).reset_index().sort_values('Work code', ascending=False),
    x='Repairman',
    y='prise',
    color='Work code',
    # size = 'duration'
    )

In [56]:
import plotly.express as px

# حذف داده‌های بدون quarter
merged_tamirat_price_bank = merged_tamirat_price_bank[~merged_tamirat_price_bank['prise'].isna()]

# تبدیل نوع داده‌ها
merged_tamirat_price_bank['quarter'] = merged_tamirat_price_bank['quarter'].astype(int)
merged_tamirat_price_bank['prise'] = merged_tamirat_price_bank['prise'].astype(int)

# 👇 تبدیل مقادیر منفی duration به مثبت (در خود دیتافریم اصلی)
merged_tamirat_price_bank['prise'] = merged_tamirat_price_bank['prise'].abs()

# گروه‌بندی و رسم
df = (
    merged_tamirat_price_bank
    .groupby(['Repairman'])
    .agg({'duration': 'sum', 'prise': 'sum', 'Work code': 'count', 'quarter': 'max'})
    .reset_index()
    .sort_values('prise', ascending=False)
)

px.scatter(
    merged_tamirat_price_bank.groupby(['Repairman']).agg({'duration': 'sum', 'prise': 'sum', 'Work code': 'count', 'quarter': 'max'}).reset_index().sort_values('prise', ascending=False),
    x='Repairman',
    y='duration',
    color='Work code',
    size = 'prise')


In [69]:


px.scatter(
    merged_tamirat_price_bank.groupby(['CompanyName']).agg({'Main system': lambda x: x.mode().iloc[0] if not x.mode().empty else None}).reset_index(),
    x='CompanyName',
    y='Main system'
)

In [70]:
merged_tamirat_price_bank.groupby(['CompanyName']).agg({'Main system': lambda x: x.mode().iloc[0] if not x.mode().empty else None}).reset_index().to_excel('cleaned_data/mode_subsystem_by_company.xlsx')

### Corrolation Analysis

In [72]:
merged_tamirat_price_bank.columns

Index([            'Work code',          'order number',
                 'Repair date',       'Machine names_x',
                 'Main system',          'Machine code',
                  'sub system',             'Repairman',
                'Type of work', 'سیستم برق و الکترونیک',
                        1742.0,    'تعمیرات2.4 الی 2.6',
                       'prise',                 'start',
                      'finish',    'قیمت دستور کار مهر',
                     'start-g',              'finish-g',
                    'duration',               'quarter',
                 'CompanyName',            'Unnamed: 1',
                    'Category',                  'Res1',
                'MobileNumber',             'UnitGroup',
                  'Unnamed: 6',     'RepairHourlyPrice',
                          'Id',            'Unnamed: 9',
                      'Status',              'UnitName',
               'LastWorkValue',           'Unnamed: 13',
                  'DriverName',

In [75]:
corr_list = ['duration', 'quarter', 'prise']
merged_tamirat_price_bank[corr_list].corr()

,duration,quarter,prise
duration,1.000000,0.020593,0.074315
quarter,0.020593,1.000000,-0.035781
prise,0.074315,-0.035781,1.000000
